<a href="https://colab.research.google.com/github/adalau05/Hello/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#perform google search, then put all url in a list
from googlesearch import search
query = '"Lu Chi Su"'#put last name at back
url_list=[] #define a list called url_list
for i in search(query, lang="en", tld="com", num=10, stop=20, pause=2):
  if (i[-3:] != "pdf"): #ignore pdf websites
    #print(i)
    url_list.append(i) #append url to url_list

In [ ]:
#save the url list to excel
import pandas as pd
url_df = pd.DataFrame([url_list])
url_df1 = url_df.transpose()
url_df1.columns = ['url']
url_df1.to_excel("url.xlsx")

In [ ]:
pip install trafilatura

In [ ]:
#read url.xlsx file
import pandas as pd
df = pd.read_excel('url.xlsx')
df.columns = ["num", "url"]

In [ ]:
#iterate through urls, extract contents and save to txt file, took 7 mins for all 60 urls
import trafilatura

j=0 #integer for incrementing filenames

for i in range(len(df["url"])) : #iterate url dataframe
  try:
    downloaded = trafilatura.fetch_url(df["url"][i]) #open url
    text = trafilatura.extract(downloaded)  #extract content from url if possible
    #print(text)
    with open("content_%s.txt" % j, 'w') as f: #create content file
      f.write(text) #save content to txt file
  except:
    print("Sorry! Cannot extract content from this webpage.") #print on screen if cannot extract content of url
  j = j+1 #iterate for next filename

Sorry! Cannot extract content from this webpage.
Sorry! Cannot extract content from this webpage.


In [ ]:
#iterate through content txt files, perform ner, find position of occurrence and word count frequency, and save to excel
import spacy
nlp = spacy.load('en_core_web_sm')

k=0 #iterate content files
frequency_list = []
for k in range(len(df["url"])) :
  try:
    with open('content_%s.txt' % k) as f:
      text = f.read()
      doc = nlp(text)
      for ent in doc.ents:
        s = [ent.text, ent.label_, ent.start_char, text.count(ent.text), k]
        frequency_list.append(s)
      #print(frequency_list)
      frequency_df = pd.DataFrame(frequency_list,columns=['Text', "Part_of_speech", "position", "frequency", "url_index"])
      frequency_df.to_excel("frequency.xlsx")
  except:
    print("Sorry! File not found.")
  k=k+1

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#define ner and tokenization library and functions

class EntityRetokenizeComponent:
  def __init__(self, nlp):
    pass
  def __call__(self, doc):
    with doc.retokenize() as retokenizer:
        for ent in doc.ents:
            retokenizer.merge(doc[ent.start:ent.end])
    return doc

nlp = spacy.load('en')
retokenizer = EntityRetokenizeComponent(nlp)
nlp.add_pipe(retokenizer, name='merge_phrases', last=True)

#open content txt files, tokenize name entities, remove stop words, find part of speech and save results to excel

from spacy.lang.en.stop_words import STOP_WORDS

k=0 #iterate content files
ner_list=[] #create ner list

for k in range(len(df["url"])) :
  try:
    with open('content_%s.txt' % k) as f:
      contents = f.read()
      doc = nlp(contents)

      tokenized_text = [tok for tok in doc if not tok.is_stop and not tok.is_punct]
      #print(tokenized_text)

      for token in tokenized_text:
        t=[token.text, token.pos_, token.idx, k]
        #print(t)
        ner_list.append(t)
  except:
    print("Sorry! File not found")
  k = k+1

In [ ]:
#save result to excel
ner_df = pd.DataFrame(ner_list)
ner_df.columns = ['Text', 'Part_of_Speech', 'Position', "url_index"]
ner_df.to_excel('ner_pos.xls')

In [ ]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=ee96cf13fb083a8904db028f060619ce5592009209a85fdab43623377fe24768
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
import pandasql as ps

#to read table
df=pd.read_excel("ner_pos.xls")

df['prev_Text'] = df['Text'].shift(-1)
df['next_Text'] = df['Text'].shift(1)

#run sql on dataframe
output_df  = ps.sqldf("SELECT prev_text,Text,next_Text,url_index from df WHERE Text = 'wife'")

df2=pd.read_excel("url.xlsx")

join_url = output_df.join(df2, on="url_index")
join_url

result = ps.sqldf("SELECT prev_text,Text,next_Text,url from join_url")
result

writer = pd.ExcelWriter('result.xlsx')
result.to_excel(writer)
writer.save()